# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [81]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [82]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [83]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox_token")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [84]:
# Import the CSVs to Pandas DataFrames
file_path = Path("toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [115]:
# Getting the data from the top 10 expensive neighbourhoods
top_10_neighbourhoods = to_data.groupby('neighbourhood').mean().nlargest(10, 'average_house_value')
top_10_neighbourhoods.head(10)

,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
neighbourhood,,,,,,,,,,,
Bridle Path-Sunnybrook-York Mills,2260.00,331.25,0.00,36.25,90.00,25.0,40.00,0.00,1526485.75,2360.75,2321.75
Forest Hill South,1742.50,2031.25,1.25,61.25,45.00,75.0,1027.50,3.75,1195992.50,1781.00,1313.75
Lawrence Park South,3472.50,773.75,0.00,126.25,38.75,225.0,966.25,16.25,1094027.75,1954.00,1372.75
Rosedale-Moore Park,2498.75,4641.25,0.00,486.25,245.00,327.5,1618.75,2.50,1093640.00,1909.75,1537.25
St.Andrew-Windfields,3225.00,1670.00,0.00,185.00,552.50,97.5,586.25,5.00,999107.00,1880.25,1384.50
Casa Loma,916.25,2310.00,0.00,288.75,201.25,162.5,1192.50,2.50,981064.25,1873.75,1547.75
Bedford Park-Nortown,4865.00,1981.25,0.00,43.75,57.50,287.5,1275.00,88.75,930415.25,1786.75,1255.00
Forest Hill North,1488.75,3392.50,0.00,12.50,16.25,82.5,402.50,1.25,851680.50,1722.75,1245.50
Kingsway South,2326.25,576.25,0.00,66.25,48.75,20.0,336.25,2.50,843234.25,1736.75,1622.00


In [86]:
# Calculate the mean number of dwelling types units per year
dwelling_totals_df = to_data.drop(['average_house_value', 'shelter_costs_owned', 'shelter_costs_rented'], axis=1)
dwelling_of_dwelling_per_year = dwelling_totals_df.groupby('year').mean()
dwelling_of_dwelling_per_year.reset_index(inplace=True)
dwelling_totals_df.head()

,neighbourhood,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house
year,,,,,,,,,
2001,Agincourt North,3715,1480,0,1055,1295,195,185,5
2001,Agincourt South-Malvern West,3250,1835,0,545,455,105,425,0
2001,Alderwood,3175,315,0,470,50,185,370,0
2001,Annex,1060,6090,5,1980,605,275,3710,165
2001,Banbury-Don Mills,3615,4465,0,240,380,15,1360,0


In [87]:
# Calculate the average monthly shelter costs for owned and rented dwellings
to_data_yr = to_data.groupby('year').mean()
cost_average_shelter_df = to_data_yr.drop(['single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house', 'average_house_value'], axis=1)
cost_average_shelter_df.head()

,shelter_costs_owned,shelter_costs_rented
year,,
2001,846.878571,1085.935714
2006,1316.800000,925.414286
2011,1448.214286,1019.792857
2016,1761.314286,1256.321429


In [89]:
to_data_yr = to_data.groupby('year').mean()
#Single out average_house_value
average_house_value_df = to_data_yr.drop(['single_detached_house', 'apartment_five_storeys_plus','movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house','shelter_costs_owned', 'shelter_costs_rented'], axis=1)
average_house_value_df.head()

,average_house_value
year,
2001,289882.885714
2006,424059.664286
2011,530424.721429
2016,664068.328571


In [90]:
#calculate the mean values for each neighbourhood
mean_value = to_data.groupby(by="neighbourhood").mean()
mean_value.reset_index(inplace=True)
mean_value = mean_value.loc[:,['neighbourhood','average_house_value']]
mean_value.head()

,neighbourhood,average_house_value
0,Agincourt North,329811.5
1,Agincourt South-Malvern West,334189.0
2,Alderwood,427922.5
3,Annex,746977.0
4,Banbury-Don Mills,612039.0


### Panel Visualization Functions

In [106]:
# Define Panel visualization functions
mean_neighbourhood_values_df = to_data[['neighbourhood', 'average_house_value']].groupby('neighbourhood').mean()
neighbourhood_locations_df = pd.read_csv(file_path)
neighbourhood_locations = neighbourhood_locations_df.set_index('neighbourhood')
mean_values_location = pd.concat([mean_neighbourhood_values_df, neighbourhood_locations_df], axis='columns', join='inner')
mean_neighbourhood_values = to_data[['neighbourhood', 'average_house_value']].groupby('neighbourhood').mean()
mean_neighbourhood_values.head()

,average_house_value
neighbourhood,
Agincourt North,329811.5
Agincourt South-Malvern West,334189.0
Alderwood,427922.5
Annex,746977.0
Banbury-Don Mills,612039.0


In [143]:
def create_bar_chart(data, title, xlabel, ylabel, color):
    
    data = data.plot(
        kind-'bar',
        rot=90,
        title=title,
        xlabel='Dwelling Types',
        ylabel="Average Monthly Cost",
        color=color
    );
    return data
   

In [96]:
def create_line_chart(data, title, xlabel, ylabel, color):
        
    data = data.plot(
        kind='line',
        rot=90,
        title=title,
        xlabel='Year',
        ylabel="Average Monthly Cost",
        color=color
    );
    return data

In [48]:
    
def average_house_value():
    """Average house values per year."""
    data = data.px.scatter(
        average_value_neighbourhood_locations,
        x="Cost of Living Index",
        y="Average Sale Price",
        size="Number of Housing Units Sold",
        color="County",
    )
    return data

In [116]:
def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""

    top_10_neighbourhoods.hvplot.bar(x='neighbourhood', y='average_house_value', rot=90, width=1000)
    return bar_chart

In [53]:
def number_dwelling_types():
    """Number of dwelling types per year"""
    number_dwelling_types = px.parallel_coordinates(number_of_dwelling_per_yr, color='index')
    return number_dwelling_types

In [146]:
def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""

    mean_value = to_data['average_house_value'].groupby('year').mean()
    mean_neighbourhood_values = to_data[['neighbourhood', 'average_house_value']]
    mean_neighbourhood_values.columns = ['Neighbourhood', 'Average House Value']
    mean_neighbourhood_values.index.names = ['Year']
    return mean_neighbourhood_values.head()

In [148]:
average_house_value_snapshot.hvplot()

AttributeError: 'function' object has no attribute 'hvplot'

In [123]:
    mean_values_location.reset_index(inplace=True)
    mapbox_values_location = px.scatter_mapbox(
        mean_values_location, 
        lat='lat', 
        lon='lon', 
        color='average_house_value', 
        size='average_house_value', 
        color_continuous_scale=px.colors.cyclical.IceFire,
        title='Average House Values in Toronto',
        hover_data={'neighbourhood': True},
        zoom=9,
        height=750,
        )
    return mapbox_values_location

ValueError: cannot insert level_0, already exists

In [26]:
def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!

AttributeError: type object 'map' has no attribute 'show'

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [136]:
#Define a title 

title = pn.pane.Markdown(
'''
# Toronto Real Estate Analysis
'''
)

In [137]:
# Define a welcome text

welcome = pn.pane.Markdown(
'''
This dashboard provides a visual analysis of the real estate market in Toronto from 2001 to 2016
'''
)

In [140]:
# Create a tab layout for the dashboard


NameError: name 'welcome_text' is not defined

In [141]:
# Create the main dashboard
dashboard  = pn.Row(title)
dashboardq = pn.Row(welcome)
dashboardr = pn.Row(map_1, chart_5)
dashboards = pn.Row(chart_2)
dashboardt = pn.Row(chart_3)
dashboardv = pn.Row(chart_4)
dashboardw = pn.Row(chart_6)
pn.Column(dashboard,dashboardq, dashboardr, dashboards, dashboardt, dashboardv, dashboardw)

NameError: name 'map_1' is not defined

## Serve the Panel Dashboard

In [135]:
dashboard.servable()
dashboard.servable()
dashboardq.servable()
dashboardr.servable()
dashboards.servable()
dashboardt.servable()
dashboardv.servable()
dashboardw.servable()
#dashboardr.servable()

NameError: name 'dashboard' is not defined

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [6]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
#create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

NameError: name 'create_bar_chart' is not defined

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()